In [1]:
import torch
import MiniFL as mfl

In [2]:
def run_client(i:int, client: mfl.algorithms.gd.GDClient):
    for _ in range(100):
        _ = client.step()
        
def run_master(master: mfl.algorithms.gd.GDMaster):
    for i in range(100):
        print(f"Master: {master.step()}")

In [6]:
from sklearn.datasets import load_svmlight_file

NUM_CLIENTS = 20

data, labels = load_svmlight_file("phishing.txt")
enc_labels = labels.copy()
data_dense = data.todense()

eval_data = (torch.from_numpy(data_dense).to(torch.float32), torch.from_numpy(enc_labels).to(torch.float32)[:, None])
clients_data = [(x, y) for x, y in zip(torch.split(eval_data[0], len(eval_data[0]) // NUM_CLIENTS, dim=0), torch.split(eval_data[1], len(eval_data[1]) // NUM_CLIENTS, dim=0))]

model = torch.nn.Linear(eval_data[0].shape[1], 1, bias=False)
loss_fn = torch.nn.BCEWithLogitsLoss()

master, clients = mfl.algorithms.gd.get_gd_master_and_clients(
    gamma=5,
    clients_data=clients_data,
    eval_data=eval_data,
    model=model,
    loss_fn=loss_fn,
)

In [7]:
import threading

client_threads = []
for i, client in enumerate(clients):
    client_threads.append(threading.Thread(target=run_client, args=(i, client)))
    client_threads[-1].start()
    
master_thread = threading.Thread(target=run_master, args=(master,))
master_thread.start()

master_thread.join()
for t in client_threads:
    t.join()
    


Master: 0.6952351331710815
Master: 0.6943375468254089
Master: 0.6934394836425781
Master: 0.6925405859947205
Master: 0.6916408538818359
Master: 0.6907404661178589
Master: 0.6898391842842102
Master: 0.6889371275901794
Master: 0.6880343556404114
Master: 0.687130868434906
Master: 0.6862267255783081
Master: 0.6853218674659729
Master: 0.6844167113304138
Master: 0.683510959148407
Master: 0.6826048493385315
Master: 0.6816985011100769
Master: 0.6807920336723328
Master: 0.6798854470252991
Master: 0.6789789795875549
Master: 0.6780725121498108
Master: 0.6771662831306458
Master: 0.6762604117393494
Master: 0.6753549575805664
Master: 0.6744500398635864
Master: 0.673545777797699
Master: 0.6726422309875488
Master: 0.6717395186424255
Master: 0.6708377599716187
Master: 0.669937014579773
Master: 0.669037401676178
Master: 0.6681390404701233
Master: 0.6672418117523193
Master: 0.6663461923599243
Master: 0.6654519438743591
Master: 0.6645591855049133
Master: 0.6636682152748108
Master: 0.6627788543701172
Master

In [8]:
clients[0].data_sender.n_bits_passed

217600